In [1]:
import pandas as pd
import datetime as dt

# Construction cost index

In [2]:
construction_file = "Files/Construction/Quarterly, Building construction prices rose, due to Homebuilder grants and government infrastructure investment.xlsx"
df_cons = pd.read_excel(construction_file,header=1,usecols="A:B", skipfooter=2)
df_cons.columns=['date','constr_index']

In [3]:
# --convert to datetime
df_cons['date'] = pd.to_datetime(df_cons['date'],format='%b-%y')

# --get year and quarter, --concatenate as time_period format, --drop other columns
df_cons['year'] = df_cons.date.dt.year
df_cons['quarter'] = df_cons.date.dt.quarter
df_cons['time_period'] = df_cons.year.map(str) + " Q" + df_cons.quarter.map(str)
df_cons_clean = df_cons.drop(columns=['date','year','quarter'],axis=1)
df_cons_clean.head()

,constr_index,time_period
0,100.1,2012 Q2
1,100.3,2012 Q3
2,100.2,2012 Q4
3,101.0,2013 Q1
4,101.6,2013 Q2


#### Joint statement:
replace the two xxx with master dataframe

xxx = pd.merge(xxx, df_cons_clean, on='time_period',how='left')

# Population

LGA to Postcode Mapping

In [4]:
SuburbLGA = "Files/Area/Postcode_and_LGA.xlsx"
postcodeLGA = pd.read_excel(SuburbLGA, usecols = "A, C, D")

postcodeLGA = postcodeLGA.dropna()
postcodeLGA["postcode"] = postcodeLGA["postcode"].astype(int)
postcodeLGA.head()

,lganame,suburbname,postcode
0,ALBURY CITY,ALBURY,2640
1,ALBURY CITY,EAST ALBURY,2640
2,ALBURY CITY,ETTAMOGAH,2640
3,ALBURY CITY,GLENROY,2640
4,ALBURY CITY,HAMILTON VALLEY,2641


In [5]:
postcodeLGA["lganame"] = postcodeLGA.lganame.str.title()

__Household per LGA, 2016 and 2021__

In [6]:
popfile = "Files/Population/2019 NSW Population Projections ASGS 2019 LGA.xlsx"
df_hhold = pd.read_excel(popfile,sheet_name='LGA Household Totals',header=6,usecols="A:C",skipfooter=3)

# --convert wide to long format with melt, --rename cols, --clean LGA name
df_hhold = pd.melt(df_hhold,id_vars='Counting households',value_vars=[2016,2021])
df_hhold.columns=['LGA','year','hhold_count']
df_hhold['LGA'] = df_hhold.LGA.str.split('(').str.get(0)
df_hhold.sample(n=10)

,LGA,year,hhold_count
105,Sydney,2016,102917
254,Wollondilly,2021,18402
8,Berrigan,2016,3880
151,Canada Bay,2021,38031
95,Randwick,2016,57003
175,Goulburn Mulwaree,2021,12880
205,Mosman,2021,12544
139,Bland,2021,2626
110,Tweed,2016,39153
7,Bellingen,2016,5562


Check LGA Map name vs household count LGA Name

In [7]:
lgamap = pd.Series(postcodeLGA.lganame.unique())
lgadf = pd.Series(df_hhold.LGA.unique())

lgacomps = pd.concat([lgamap,lgadf],axis=1)
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(lgacomps)

                                          0                               1
0                               Albury City                         Albury 
1                         Armidale Regional              Armidale Regional 
2                                   Ballina                        Ballina 
3                                 Balranald                      Balranald 
4                         Bathurst Regional              Bathurst Regional 
5                                   Bayside                        Bayside 
6                               Bega Valley                    Bega Valley 
7                                 Bellingen                      Bellingen 
8                                  Berrigan                       Berrigan 
9                                 Blacktown                      Blacktown 
10                                    Bland                          Bland 
11                                  Blayney                        Blayney 
12          